# HR Deep Agent - TechCorp India

## Overview
Production-ready HR Deep Agent using LangChain's Deep Agents architecture:

- **Planning Tool**: Breaks down complex HR workflows into manageable subtasks
- **Virtual Filesystem**: Stores candidate profiles, reports, and intermediate results
- **Sub-agents**: Delegates specialized tasks to focused agents
- **Indian Context**: Handles CTC structure, Indian locations, and HR practices

### What is a Deep Agent?

Traditional agents call tools in a loop but fail on complex, multi-step tasks. Deep Agents solve this with:

1. **Planning**: Built-in `write_todos` tool for task decomposition
2. **Persistent State**: File system tools for managing large documents
3. **Sub-agents**: `task` tool for delegating specialized work
4. **Detailed Prompts**: Domain-specific instructions for HR operations

### Architecture

```
User Query → Deep Agent → [Planning] → [Execute Tools] → [Sub-agents] → Response
                              ↓            ↓               ↓
                          TODO List    HR Tools      Specialist Agents
                                       (10 tools)     (context isolated)
```

## Setup and Installation

In [ ]:
# Install required packages
!pip install -q deepagents langchain-openai langchain-core python-dotenv

In [ ]:
# Import libraries
import os
import json
from typing import Optional, List, Literal, Dict, Any
from datetime import datetime
from dotenv import load_dotenv

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# Deep Agents import
from deepagents import create_deep_agent

# Load environment variables
load_dotenv()

# Verify OpenAI API key
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("⚠️ Please set OPENAI_API_KEY in your environment")

print("✅ Setup complete!")
print(f"📅 Current date: {datetime.now().strftime('%Y-%m-%d')}")

## HR Database (Mock Data with Indian Context)

In [ ]:
# Mock HR Database for TechCorp India
HR_DATABASE = {
    "candidates": {
        "CAN001": {
            "name": "Priya Sharma",
            "email": "priya.sharma@email.com",
            "phone": "+91-98765-43210",
            "position": "Senior Backend Engineer",
            "status": "screening",
            "experience_years": 6,
            "skills": ["Python", "FastAPI", "React", "AWS", "Docker", "Kubernetes"],
            "current_ctc": "₹22 LPA",
            "expected_ctc": "₹30 LPA",
            "location": "Bengaluru, Karnataka",
            "notice_period": "60 days"
        },
        "CAN002": {
            "name": "Arjun Mehta",
            "email": "arjun.mehta@email.com",
            "phone": "+91-98123-45678",
            "position": "Senior Backend Engineer",
            "status": "interview_scheduled",
            "experience_years": 7,
            "skills": ["Java", "Spring Boot", "Microservices", "AWS", "Kafka"],
            "current_ctc": "₹25 LPA",
            "expected_ctc": "₹32 LPA",
            "location": "Pune, Maharashtra",
            "notice_period": "30 days"
        },
        "CAN003": {
            "name": "Kavya Reddy",
            "email": "kavya.reddy@email.com",
            "phone": "+91-97654-32109",
            "position": "Frontend Developer",
            "status": "screening",
            "experience_years": 4,
            "skills": ["React", "TypeScript", "Next.js", "TailwindCSS"],
            "current_ctc": "₹15 LPA",
            "expected_ctc": "₹20 LPA",
            "location": "Hyderabad, Telangana",
            "notice_period": "45 days"
        }
    },
    "employees": {
        "EMP001": {
            "name": "Rahul Verma",
            "role": "Engineering Manager",
            "email": "rahul.verma@techcorp.in",
            "department": "Engineering",
            "location": "Bengaluru"
        },
        "EMP002": {
            "name": "Anjali Patel",
            "role": "HR Manager",
            "email": "anjali.patel@techcorp.in",
            "department": "Human Resources",
            "location": "Mumbai"
        },
        "EMP003": {
            "name": "Vikram Singh",
            "role": "Tech Lead",
            "email": "vikram.singh@techcorp.in",
            "department": "Engineering",
            "location": "Bengaluru"
        }
    },
    "interview_slots": [
        {"date": "2025-10-25", "time": "10:00 AM", "interviewer": "Rahul Verma", "available": True, "type": "Technical"},
        {"date": "2025-10-25", "time": "2:00 PM", "interviewer": "Vikram Singh", "available": True, "type": "Technical"},
        {"date": "2025-10-26", "time": "11:00 AM", "interviewer": "Anjali Patel", "available": True, "type": "HR"},
        {"date": "2025-10-26", "time": "3:00 PM", "interviewer": "Rahul Verma", "available": True, "type": "Managerial"}
    ]
}

print(f"✅ HR Database loaded:")
print(f"   📋 {len(HR_DATABASE['candidates'])} candidates")
print(f"   👥 {len(HR_DATABASE['employees'])} employees")
print(f"   📅 {len(HR_DATABASE['interview_slots'])} interview slots")

## Define HR Tools

In [ ]:
@tool
def search_candidates(
    position: Optional[str] = None,
    min_experience: Optional[int] = None,
    location: Optional[str] = None
) -> str:
    """Search for candidates by position, experience, or location."""
    results = []
    for cid, cand in HR_DATABASE["candidates"].items():
        match = True
        if position and position.lower() not in cand["position"].lower():
            match = False
        if min_experience and cand["experience_years"] < min_experience:
            match = False
        if location and location.lower() not in cand["location"].lower():
            match = False
        if match:
            results.append({
                "id": cid,
                "name": cand["name"],
                "position": cand["position"],
                "experience": f"{cand['experience_years']} years",
                "location": cand["location"],
                "status": cand["status"]
            })
    return json.dumps(results, indent=2) if results else "No candidates found."


@tool
def get_candidate_details(candidate_id: str) -> str:
    """Get complete profile of a specific candidate."""
    cand = HR_DATABASE["candidates"].get(candidate_id)
    return json.dumps(cand, indent=2) if cand else f"Candidate {candidate_id} not found."


@tool
def check_interview_availability(
    interviewer_name: Optional[str] = None,
    date: Optional[str] = None
) -> str:
    """Check available interview slots, optionally filtered by interviewer or date."""
    slots = [s for s in HR_DATABASE["interview_slots"] if s["available"]]
    if interviewer_name:
        slots = [s for s in slots if interviewer_name.lower() in s["interviewer"].lower()]
    if date:
        slots = [s for s in slots if date in s["date"]]
    return json.dumps(slots, indent=2) if slots else "No available slots found."


@tool
def schedule_interview(
    candidate_id: str,
    date: str,
    time: str,
    interviewer_name: str
) -> str:
    """Schedule an interview for a candidate."""
    cand = HR_DATABASE["candidates"].get(candidate_id)
    if not cand:
        return f"❌ Candidate {candidate_id} not found."
    
    # Find and book the slot
    for slot in HR_DATABASE["interview_slots"]:
        if (slot["date"] == date and slot["time"] == time and 
            interviewer_name.lower() in slot["interviewer"].lower() and slot["available"]):
            slot["available"] = False
            HR_DATABASE["candidates"][candidate_id]["status"] = "interview_scheduled"
            return f"""✅ Interview scheduled!
Candidate: {cand['name']} ({candidate_id})
Date: {date} at {time}
Interviewer: {slot['interviewer']}
Type: {slot['type']} Round

📧 Calendar invites sent."""
    
    return f"❌ Slot not available: {date} at {time} with {interviewer_name}"


@tool
def update_candidate_status(candidate_id: str, new_status: str) -> str:
    """Update candidate status (screening/interview_scheduled/offer/rejected)."""
    cand = HR_DATABASE["candidates"].get(candidate_id)
    if not cand:
        return f"❌ Candidate {candidate_id} not found."
    old_status = cand["status"]
    cand["status"] = new_status
    return f"✅ {cand['name']}: {old_status} → {new_status}"


@tool
def calculate_ctc_breakdown(ctc_lpa: int) -> str:
    """Calculate Indian CTC breakdown with monthly in-hand estimate."""
    base = ctc_lpa * 0.65
    hra = base * 0.40
    special = ctc_lpa * 0.15
    bonus = ctc_lpa * 0.15
    monthly_gross = (base + hra + special) / 12
    monthly_inhand = monthly_gross * 0.85  # Approx after tax
    
    return f"""💰 CTC Breakdown for ₹{ctc_lpa} LPA

Annual Components:
━━━━━━━━━━━━━━━━━━━━━━━━━━━
Base Salary:        ₹{base:.2f} L
HRA (40% of base):  ₹{hra:.2f} L
Special Allowance:  ₹{special:.2f} L
Performance Bonus:  ₹{bonus:.2f} L

Monthly:
━━━━━━━━━━━━━━━━━━━━━━━━━━━
Gross Monthly:      ₹{monthly_gross:.2f} L
Approx In-hand:     ₹{monthly_inhand:.2f} L
(After tax deductions)
"""


@tool
def send_email(to_email: str, subject: str, body: str) -> str:
    """Send email notification."""
    return f"""✅ Email sent!
To: {to_email}
Subject: {subject}
[SIMULATED - Would send in production]"""


# Collect all tools
hr_tools = [
    search_candidates,
    get_candidate_details,
    check_interview_availability,
    schedule_interview,
    update_candidate_status,
    calculate_ctc_breakdown,
    send_email
]

print("✅ HR Tools defined:")
for t in hr_tools:
    print(f"   • {t.name}")

## Create the HR Deep Agent

In [ ]:
# Custom system prompt for HR domain
HR_SYSTEM_PROMPT = """You are an expert HR Assistant at TechCorp India.

Your expertise:
- Talent acquisition and candidate screening
- Interview coordination and scheduling
- Indian HR practices (CTC structure, leave policies, notice periods)
- Professional communication with candidates and hiring managers

When handling complex tasks:
1. Use write_todos to break down multi-step workflows
2. Store candidate profiles and reports in files (write_file, read_file)
3. Delegate specialized subtasks to sub-agents (task tool)
4. Provide clear, professional responses

Always maintain confidentiality and provide excellent candidate experience."""

# Initialize OpenAI model
model = ChatOpenAI(
    model="gpt-4o",
    temperature=0.2
)

# Create the Deep Agent
hr_agent = create_deep_agent(
    model=model,
    tools=hr_tools,
    system_prompt=HR_SYSTEM_PROMPT
)

print("✅ HR Deep Agent created!")
print(f"   🤖 Model: OpenAI GPT-4o")
print(f"   🔧 Tools: {len(hr_tools)}")
print(f"   🧠 Built-in: Planning, Filesystem, Sub-agents")

## Helper Function for Running Queries

In [ ]:
def run_hr_query(query: str, verbose: bool = False):
    """Execute a query with the HR Deep Agent."""
    print(f"\n{'='*80}")
    print(f"🔍 Query: {query}")
    print(f"{'='*80}\n")
    
    result = hr_agent.invoke({"messages": [{"role": "user", "content": query}]})
    
    if verbose:
        print("\n📊 Full trace:")
        for msg in result["messages"]:
            print(f"  {type(msg).__name__}: {getattr(msg, 'content', 'N/A')[:100]}...")
    
    print("\n💬 Agent Response:")
    print("-" * 80)
    
    # Extract final response
    for msg in reversed(result["messages"]):
        if hasattr(msg, "content") and msg.content and not hasattr(msg, "tool_calls"):
            print(msg.content)
            break
    
    print("\n" + "="*80)
    return result

print("✅ Helper function ready!")

---

# Test Cases

## Test 1: Simple Candidate Search

In [ ]:
# Simple search query
run_hr_query(
    "Find all candidates applying for Backend Engineer positions in Bengaluru"
)

## Test 2: Candidate Profile Review

In [ ]:
# Get detailed candidate information
run_hr_query(
    "Show me the complete profile for candidate CAN001 (Priya Sharma). Is she a good fit for Senior Backend Engineer role?"
)

## Test 3: Interview Scheduling

In [ ]:
# Check availability and schedule
run_hr_query("""
I need to schedule interviews for Priya Sharma (CAN001):
1. Check what slots are available with Rahul Verma
2. Schedule a technical interview for her
3. Send confirmation email
""")

## Test 4: Complex Multi-Step Workflow

This demonstrates the Deep Agent's ability to:
- Plan multiple steps
- Use multiple tools in sequence
- Synthesize information from different sources

In [ ]:
# Complex workflow
run_hr_query("""
Complete end-to-end hiring workflow for candidate CAN001:

1. Get her full profile and assess qualifications
2. Calculate appropriate CTC offer (15% above her expectation)
3. Find and schedule 2 interview rounds:
   - Technical with Rahul Verma
   - HR with Anjali Patel
4. Update her status to interview_scheduled
5. Draft an email with:
   - Interview schedule
   - CTC offer indication
   - Next steps

Provide a complete summary at the end.
""")

## Test 5: Compensation Analysis

In [ ]:
# CTC breakdown
run_hr_query(
    "Calculate and explain the CTC breakdown for ₹35 LPA. What would be the monthly in-hand salary?"
)

## Test 6: Bulk Processing (Demonstrates Sub-agents)

In [ ]:
# Process multiple candidates
run_hr_query("""
Create a candidate comparison report for all Backend Engineer applicants:

For each candidate (CAN001, CAN002):
1. Get their profile
2. List key qualifications
3. Calculate recommended offer (12% above expectation)
4. Assess cultural fit based on location preference

Then create a ranked recommendation with justification.
Use the file system to store individual assessments.
""")

---

## Understanding the Deep Agent

### What Just Happened?

The Deep Agent automatically:

1. **Planned** the workflow using `write_todos`
2. **Executed** tools in the right sequence
3. **Stored** large results in the virtual filesystem
4. **Delegated** complex subtasks to sub-agents
5. **Synthesized** a coherent final response

### Built-in Tools (You don't see these but they're used)

- `write_todos`: Plan and track subtasks
- `write_file`: Store candidate profiles, reports
- `read_file`: Retrieve stored documents
- `edit_file`: Update existing documents
- `ls`: List all files in the filesystem
- `task`: Spawn sub-agent for specialized work

### Why This is "Deep"

Traditional agents would:
- Get overwhelmed by long context
- Forget earlier steps
- Not plan ahead

Deep Agents:
- Offload data to files (reduces context pressure)
- Maintain explicit TODO list (tracks progress)
- Delegate to specialists (isolates concerns)
- Plan before executing (strategic thinking)

## Custom Query - Try Your Own!

In [ ]:
# Your custom query here
custom_query = """
Find the best candidate for our Senior Backend Engineer role and schedule a complete interview loop.
"""

run_hr_query(custom_query)

---

## Next Steps

### Production Deployment

1. **Replace mock data** with real database connections
2. **Add authentication** for secure access
3. **Integrate with ATS** (Greenhouse, Lever, etc.)
4. **Connect email service** (SendGrid, AWS SES)
5. **Add audit logging** for compliance

### Customization

```python
# Add new tools
@tool
def background_verification(candidate_id: str) -> str:
    """Check background verification status."""
    # Your implementation
    pass

hr_tools.append(background_verification)

# Recreate agent with updated tools
hr_agent = create_deep_agent(
    model=model,
    tools=hr_tools,
    system_prompt=HR_SYSTEM_PROMPT
)
```

### Resources

- **Deep Agents Docs**: https://docs.langchain.com/oss/python/deepagents/
- **LangChain Docs**: https://docs.langchain.com/
- **OpenAI API**: https://platform.openai.com/docs/
- **Reference**: https://github.com/cloudyuga/agentic-ai-w-mcp/